okay so here I want to dive a bit on grp servers and clients. Especially when it comes to streaming server/client and the issues related to parallel execuation etc.


## Setup a gRPC Service/ Client in Python

okay so this should descrive a basic unary grpc client/server. I guess I scartched this topic a bit in previous run.
But didn't put a compelete example.
Unary RPCs:- This is a simple gRPC which works like a normal function call. It sends a single request declared in the .proto file to the server and gets back a single response from the server.

TBC




first start with this step by step [guide](https://velotio.medium.com/implementing-grpc-in-python-a-step-by-step-guide-e9733871acb0)


So the example starts with virtualenv for env setup:

```
mkdir grpc_example
cd grpc_example
virtualenv -p python3 env
source env/bin/activate
pip install grpcio grpcio-tools
```

I am going to try with docker instead.

So I need a dockerfile
    intall python, etc.
    install grpc tools
    add the example proto file
    compile the proto file

then I add the example server:

```
import grpc
from concurrent import futures
import time
import unary_pb2_grpc as pb2_grpc
import unary_pb2 as pb2


class UnaryService(pb2_grpc.UnaryServicer):

    def __init__(self, *args, **kwargs):
        pass

    def GetServerResponse(self, request, context):

        # get the string from the incoming request
        message = request.message
        result = f'Hello I am up and running received "{message}" message from you'
        result = {'message': result, 'received': True}

        return pb2.MessageResponse(**result)


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    pb2_grpc.add_UnaryServicer_to_server(UnaryService(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    serve()
```


and the client code :

```
import grpc
import unary_pb2_grpc as pb2_grpc
import unary_pb2 as pb2


class UnaryClient(object):
    """
    Client for gRPC functionality
    """

    def __init__(self):
        self.host = 'localhost'
        self.server_port = 50051

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.UnaryStub(self.channel)

    def get_url(self, message):
        """
        Client function to call the rpc for GetServerResponse
        """
        message = pb2.Message(message=message)
        print(f'{message}')
        return self.stub.GetServerResponse(message)


if __name__ == '__main__':
    client = UnaryClient()
    result = client.get_url(message="Hello Server you there?")
    print(f'{result}')


```

So my first comaand line for the compiling the proto did not work quite well:

```
python -m grpc_tools.protoc  -I . --python_out=. --grpc_python_out=.
```

I had to change it like this:
```
python3 -m grpc_tools.protoc --proto_path=. ./unary.proto --python_out=. --grpc_python_out=.
```
(okay looks like in the first one I just forgot to place the proto file name, i.e. ./unary.proto)

and finally in order to call both server and client in the same container, I needed to call the server in the detached mode:

so I used:

```
python3 server.py &
```



## Setup Streaming Server/Client



4. Bidirectional streaming RPCs:- Both gRPC client and the gRPC server use a read-write stream to send a message sequence. Both operate independently, so gRPC clients and gRPC servers can write and read in any order they like, i.e. the server can read a message then write a message alternatively, wait to receive all messages then write its responses, or perform reads and writes in any other combination.

```
rpc HelloServer(stream RequestMessage) returns (stream ResponseMessage);
```


**note** gRPC guarantees the ordering of messages within an individual RPC call. In the case of Bidirectional streaming, the order of messages is preserved in each stream.
(Does it mean that we really don't have to worry about sorting which response belobed to which request??! I'm not sure yet)


So I created the example for the streaming in a seperate directory (example2) but I could have had the same docker file extended to have both unary and streaming. ANyways...

** Note** The name of proto file should be the same as the package we import in server/client codes. This would be more obvious if we compile the proto first and work diretly with python modules.

This is the server code:

```
from concurrent import futures

import grpc
import bidirectional_pb2_grpc as bidirectional_pb2_grpc


class BidirectionalService(bidirectional_pb2_grpc.BidirectionalServicer):

    def GetServerResponse(self, request_iterator, context):
        for message in request_iterator:
            yield message


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    bidirectional_pb2_grpc.add_BidirectionalServicer_to_server(BidirectionalService(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    serve()
```


okay the part that I searched a lot to land on, the last time I was trying to implement this, was around the way that server and client crreate and receive the iterative objects. here we use a for loop with a 'yeild' statements. 

(btw,I saw some other advanced ways then that I couldn't quite understand, now hopefully we can address them here.)


The client code:

```
from __future__ import print_function

import grpc
import bidirectional_pb2_grpc as bidirectional_pb2_grpc
import bidirectional_pb2 as bidirectional_pb2


def make_message(message):
    return bidirectional_pb2.Message(
        message=message
    )


def generate_messages():
    messages = [
        make_message("First message"),
        make_message("Second message"),
        make_message("Third message"),
        make_message("Fourth message"),
        make_message("Fifth message"),
    ]
    for msg in messages:
        print("Hello Server Sending you the %s" % msg.message)
        yield msg


def send_message(stub):
    responses = stub.GetServerResponse(generate_messages())
    for response in responses:
        print("Hello from the server received your %s" % response.message)


def run():
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = bidirectional_pb2_grpc.BidirectionalStub(channel)
        send_message(stub)


if __name__ == '__main__':
    run()


note we could have either client only or server only as streaming as well.

## Streaming Server client - grpc official python example

so this example is from github grpc repo under python examples:

https://github.com/grpc/grpc/tree/v1.20.0/examples/python/route_guide

I read through the code but it wasn't obvious to me what is going on.

So I used this grpc [tutorial] (https://grpc.github.io/docs/tutorials/basic/python.html) that describes what is going on.


**Note** this [address]( https://grpc.io/docs/languages/python/) is great  enterance to python grpc.

This [examples](https://github.com/grpc/grpc/tree/master/examples/python) repo is also great to go through as well, sometime.


first defining the proto:

```

syntax = "proto3";

option java_multiple_files = true;
option java_package = "io.grpc.examples.routeguide";
option java_outer_classname = "RouteGuideProto";
option objc_class_prefix = "RTG";

package routeguide;

// Interface exported by the server.
service RouteGuide {
  // A simple RPC.
  //
  // Obtains the feature at a given position.
  //
  // A feature with an empty name is returned if there's no feature at the given
  // position.
  rpc GetFeature(Point) returns (Feature) {}

  // A server-to-client streaming RPC.
  //
  // Obtains the Features available within the given Rectangle.  Results are
  // streamed rather than returned at once (e.g. in a response message with a
  // repeated field), as the rectangle may cover a large area and contain a
  // huge number of features.
  rpc ListFeatures(Rectangle) returns (stream Feature) {}

  // A client-to-server streaming RPC.
  //
  // Accepts a stream of Points on a route being traversed, returning a
  // RouteSummary when traversal is completed.
  rpc RecordRoute(stream Point) returns (RouteSummary) {}

  // A Bidirectional streaming RPC.
  //
  // Accepts a stream of RouteNotes sent while a route is being traversed,
  // while receiving other RouteNotes (e.g. from other users).
  rpc RouteChat(stream RouteNote) returns (stream RouteNote) {}
}

// Points are represented as latitude-longitude pairs in the E7 representation
// (degrees multiplied by 10**7 and rounded to the nearest integer).
// Latitudes should be in the range +/- 90 degrees and longitude should be in
// the range +/- 180 degrees (inclusive).
message Point {
  int32 latitude = 1;
  int32 longitude = 2;
}

// A latitude-longitude rectangle, represented as two diagonally opposite
// points "lo" and "hi".
message Rectangle {
  // One corner of the rectangle.
  Point lo = 1;

  // The other corner of the rectangle.
  Point hi = 2;
}

// A feature names something at a given point.
//
// If a feature could not be named, the name is empty.
message Feature {
  // The name of the feature.
  string name = 1;

  // The point where the feature is detected.
  Point location = 2;
}

// A RouteNote is a message sent while at a given point.
message RouteNote {
  // The location from which the message is sent.
  Point location = 1;

  // The message to be sent.
  string message = 2;
}

// A RouteSummary is received in response to a RecordRoute rpc.
//
// It contains the number of individual points received, the number of
// detected features, and the total distance covered as the cumulative sum of
// the distance between each point.
message RouteSummary {
  // The number of points received.
  int32 point_count = 1;

  // The number of known features passed while traversing the route.
  int32 feature_count = 2;

  // The distance covered in metres.
  int32 distance = 3;

  // The duration of the traversal in seconds.
  int32 elapsed_time = 4;
}
```

So what we get from the proto?! The services and rpcs:


```
service RouteGuide {
  rpc GetFeature(Point) returns (Feature) {}

  rpc ListFeatures(Rectangle) returns (stream Feature) {}

  rpc RecordRoute(stream Point) returns (RouteSummary) {}

  rpc RouteChat(stream RouteNote) returns (stream RouteNote) {}
}
```


The service included 4 rpcs:

- A Simple RPC:
    Obtains the feature at a given position.  A feature with an empty name is returned if there's no feature at the given position.
- A server-to-client streaming RPC: 
    Obtains the Features available within the given Rectangle.  Results are streamed rather than returned at once     (e.g. in a response message        with a repeated field), as the rectangle may cover a large area and contain a huge number of features.
- A client-to-server streaming RPC:
    Accepts a stream of Points on a route being traversed, returning a RouteSummary when traversal is completed.
- A Bidirectional streaming RPC:
  Accepts a stream of RouteNotes sent while a route is being traversed, while receiving other RouteNotes (e.g. from other users).



now the proto messages:

```
message Point {
  int32 latitude = 1;
  int32 longitude = 2;
}

message Rectangle {
  // One corner of the rectangle.
  Point lo = 1;

  // The other corner of the rectangle.
  Point hi = 2;
}
```

A feature names something at a given point. If a feature could not be named, the name is empty.

```
message Feature {
  // The name of the feature.
  string name = 1;
  // The point where the feature is detected.
  Point location = 2;
}
```

A RouteNote is a message sent while at a given point.
A RouteSummary is received in response to a RecordRoute rpc.

```
message RouteNote {
  // The location from which the message is sent.
  Point location = 1;
  // The message to be sent.
  string message = 2;
}

message RouteSummary {
  // The number of points received.
  int32 point_count = 1;

  // The number of known features passed while traversing the route.
  int32 feature_count = 2;

  // The distance covered in metres.
  int32 distance = 3;

  // The duration of the traversal in seconds.
  int32 elapsed_time = 4;
}
```


next, minor updates to the docker file to compile the proto.

What the compiled files include?

- RouteGuideStub, which can be used by clients to invoke RouteGuide RPCs
- RouteGuideServicer, which defines the **interface for implementations** of the RouteGuide service.
The functions for the service defined in route_guide.proto

example:

```

class RouteGuideServicer(object):
  """Interface exported by the server.
  """

  def GetFeature(self, request, context):
    """A simple RPC.
    Obtains the feature at a given position.
    A feature with an empty name is returned if there's no feature at the given
    position.
    """
    context.set_code(grpc.StatusCode.UNIMPLEMENTED)
    context.set_details('Method not implemented!')
    raise NotImplementedError('Method not implemented!')
```

- add_RouteGuideServicer_to_server, which adds a RouteGuideServicer to a grpc.Server


**Note** I should look more on the interface to get idea on proper impelementation of the function. Specially for parallel execution which is my intent.


Next is the code for the grpc server:


so the server code should implement 

```
class RouteGuideServicer(route_guide_pb2_grpc.RouteGuideServicer):
    """Provides methods that implement functionality of route guide server."""

    def __init__(self):
        self.db = route_guide_resources.read_route_guide_database()

```

This has a module 'route_guide_resources' to import.

This module reads a json file "route_guide_db.json".

Also should include some python_requirments file to install packages.

ok so the 'self.db' will be a list of feature prto objects.

then we reach to the implementation of get_feature rpc:

```
 def GetFeature(self, request, context):
        feature = get_feature(self.db, request)
        if feature is None:
            return route_guide_pb2.Feature(name="", location=request)
        else:
            return feature
```
which uses the "get_feature" function:

```

def get_feature(feature_db, point):
    """Returns Feature at given location or None."""
    for feature in feature_db:
        if feature.location == point:
            return feature
    return None

```
it will return a feature.

so this is the unary rpc.



The second rpc was single request, streaming response:

```
def ListFeatures(self, request, context):
        left = min(request.lo.longitude, request.hi.longitude)
        right = max(request.lo.longitude, request.hi.longitude)
        top = max(request.lo.latitude, request.hi.latitude)
        bottom = min(request.lo.latitude, request.hi.latitude)
        for feature in self.db:
            if (feature.location.longitude >= left and
                    feature.location.longitude <= right and
                    feature.location.latitude >= bottom and
                    feature.location.latitude <= top):
                yield feature
```

which uses of yield again for delivering the stream.

and the third one is when reaceiving from a stream:

```
 def RecordRoute(self, request_iterator, context):
        point_count = 0
        feature_count = 0
        distance = 0.0
        prev_point = None

        start_time = time.time()
        for point in request_iterator:
            point_count += 1
            if get_feature(self.db, point):
                feature_count += 1
            if prev_point:
                distance += get_distance(prev_point, point)
            prev_point = point

        elapsed_time = time.time() - start_time
        return route_guide_pb2.RouteSummary(
            point_count=point_count,
            feature_count=feature_count,
            distance=int(distance),
            elapsed_time=int(elapsed_time))
```

I guess this one only has a "request_iterator" that is treated just like a list.
and the last one does not seem to have anything special about it.


for the client code:

```


def guide_get_one_feature(stub, point):
    feature = stub.GetFeature(point)
    if not feature.location:
        print("Server returned incomplete feature")
        return

    if feature.name:
        print("Feature called %s at %s" % (feature.name, feature.location))
    else:
        print("Found no feature at %s" % feature.location)


def guide_get_feature(stub):
    guide_get_one_feature(stub,
                          route_guide_pb2.Point(
                              latitude=409146138, longitude=-746188906))
    guide_get_one_feature(stub, route_guide_pb2.Point(latitude=0, longitude=0))

```
and


```
print("-------------- GetFeature --------------")
guide_get_feature(stub)
```

okay so the part I like here is that it passes the 'stub' as argument around and utilize the rpc function where  fits best
as well as the atribs of the response proto etc.
I guess same is true for the server.


okay now onto unary request streaming response:

```
def guide_list_features(stub):
    rectangle = route_guide_pb2.Rectangle(
        lo=route_guide_pb2.Point(latitude=400000000, longitude=-750000000),
        hi=route_guide_pb2.Point(latitude=420000000, longitude=-730000000))
    print("Looking for features between 40, -75 and 42, -73")

    features = stub.ListFeatures(rectangle)

    for feature in features:
        print("Feature called %s at %s" % (feature.name, feature.location))
```

so the response it gets from rpc is like a list. now, Are we sure the order is preserved?

okay so I guess this is pretty much it. Now the last thing to do here is to run these client and server.

and the docker container runs pretty!


## Concurrent Executaion
